# TSDC Data Cleaning

This notebook is set up to intake the files from the TSDC records and process them according to the data cleaning outlined in our paper

Current count is precise num users and up by 3 trips

In [ ]:
#path configuration
to_data_parent = "../Data/abby_ceo" #path to the parent folder, should contain program subfolders
to_mini_data = "../Data/mini_pilot/data" #path to the mini data folder, contains an analysis trips file
to_data_folder = "../Data" #data folder, where composite data files will be written/read

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from data_utilities import *

# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r df_ei
%store -r dic_re
%store -r dic_pur
%store -r dic_fuel

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_fuel = defaultdict(lambda: 'Other',dic_fuel)

## Mini Pilot Data

In [ ]:
mini_confirmed_trips = pd.read_csv(to_mini_data + '/analysis_confirmed_trip.csv')

In [ ]:
print(len(mini_confirmed_trips), "total minipilot trips")
print(mini_confirmed_trips.perno.nunique(), "total minipilot users")

In [ ]:
## remove trips with no label and count again
labeled_mini = mini_confirmed_trips[mini_confirmed_trips.data_user_input_mode_confirm.notna()]
labeled_mini = mini_confirmed_trips[mini_confirmed_trips.data_user_input_purpose_confirm.notna()]

print(len(labeled_mini), "labeled minipilot trips") #only 25 over data used in paper
print(labeled_mini.perno.nunique(), "minipilot users who labeled")#same as data used in paper

In [ ]:
mini_data = labeled_mini.copy()

#first, add the cleaned mode
mini_data['Mode_confirm']= mini_data['data_user_input_mode_confirm'].map(dic_re)

#second, add the cleaned replaced mode ASSUMES PROGRAM
mini_data['Replaced_mode']= mini_data['data_user_input_replaced_mode'].map(dic_re)

#third, add the cleaned purpose
mini_data['Trip_purpose']= mini_data['data_user_input_purpose_confirm'].map(dic_pur)

In [ ]:
# Combine variable categories
mini_data = mini_data.replace('Gas Car, drove alone', 'Car')
mini_data = mini_data.replace('Gas Car, with others', 'Shared Car')
mini_data = mini_data.replace('Bikeshare', 'Shared Micromobility')
mini_data = mini_data.replace('Scooter share', 'Shared Micromobility')
mini_data = mini_data.replace('Regular Bike', 'Personal Micromobility')
mini_data = mini_data.replace('Skate board', 'Personal Micromobility')
mini_data = mini_data.replace('Train', 'Transit')
mini_data = mini_data.replace('Free Shuttle', 'Transit')
mini_data = mini_data.replace('Bus', 'Transit')
mini_data = mini_data.replace('Walk', 'Walk')
mini_data = mini_data.replace('Taxi/Uber/Lyft', 'Ridehail')
mini_data = mini_data.replace('Pilot ebike', 'E-Bike')

#filter out 'not a trip' trips
mini_data = mini_data[~mini_data['Mode_confirm'].isin(['Not a Trip'])]
mini_data = mini_data[~mini_data['Replaced_mode'].isin(['Not a Trip'])]
mini_data = mini_data[~mini_data['Trip_purpose'].isin(['not_a_trip'])]

print(len(mini_data), "trips once not a trip is removed")

In [ ]:
mini_data.loc[mini_data['Mode_confirm']=='Personal Micromobility', 'Mode_confirm'] = 'Other'
mini_data.loc[mini_data['Mode_confirm']=='Shared Micromobility', 'Mode_confirm'] = 'Other'

all_trips = mini_data.groupby(['Mode_confirm'], as_index=False).count()[['Mode_confirm','data_distance']]
all_trips['proportion'] = all_trips['data_distance'] / np.sum(all_trips.data_distance)
all_trips['trip_type'] = 'All Trips'

work_trips = mini_data[mini_data['Trip_purpose']=='Work'].copy()
work_trips = work_trips.groupby(['Mode_confirm'], as_index=False).count()[['Mode_confirm','data_distance']]
work_trips['proportion'] = work_trips['data_distance'] / np.sum(work_trips.data_distance)
work_trips['trip_type'] = 'Work Trips'
work_trips.loc[1.5] = 'Other', 0, 0, 'Work Trips'
work_trips = work_trips.sort_index().reset_index(drop=True)

mini_data = pd.concat([all_trips,work_trips])
mini_data['Dataset'] = 'Minipilot'
mini_data.columns = ['Mode','Count','Proportion','Trip Type', "Dataset"]

In [ ]:
mini_data #trip breakdown

### matching minis to survey data

In [ ]:
mini_trips = pd.read_csv(to_mini_data + '/analysis_confirmed_trip.csv')
mini_surveys = pd.read_csv(to_mini_data + '/survey_household.csv')

print(len(mini_trips), "minpilot trips")
print(len(mini_surveys), "minpilot surveys") #15 surveys
print(mini_trips.perno.nunique(), "minpilot users") #13 unique users

socio_data = mini_surveys[~mini_surveys.perno.isnull()]
print(len(socio_data), "surveys after dropping null")

# Deal with people who have multiple responses by using most recent
socio_data = socio_data.sort_values(by=['perno', 'timestamp'])
socio_data.drop_duplicates(subset=['perno'], keep='last', inplace=True)
socio_data['user_id_socio'] = socio_data.perno
socio_data.user_id_socio = [i.replace('-','') for i in socio_data.user_id_socio] # remove all dashes from strings
socio_data = socio_data.drop(labels='perno', axis=1)

# Lose some trips due to people with no survey responses
mini_trips['user_id_socio'] = mini_trips.perno.astype(str)
mini_trips.user_id_socio = [i.replace('-','') for i in mini_trips.user_id_socio] # remove all dashes from strings
mini_trips = mini_trips.merge(socio_data, on='user_id_socio')

print(mini_trips.user_id_socio.nunique(), "minipilot users with surveys")
print(len(mini_trips), "trips after pairing with surveys")

mini_trips.to_csv(to_data_folder + "/minipilot_cleaned.csv")

## Full Pilot Data

In [ ]:
#loop over
folders = ['4c', 'cc', 'fc', 'pc', 'sc', 'vail_22']
datasets = []

for program in folders:
    print('\nstarting with ', program)
    
    #create dataset with surveys and trips
    trips = pd.read_csv(to_data_parent + '/' + program + '/analysis_confirmed_trip.csv')
    print(len(trips), 'trips')
    print(trips.perno.nunique(), 'people')

    surveys = pd.read_csv(to_data_parent + '/' + program + '/' + program + '_survey_household.csv')
    print(len(surveys), 'surveys')

    #drop any null ids
    socio_data = surveys[~surveys['unique_user_id_autofilled_do_not_edit'].isnull()]
    print(len(socio_data), 'surveys after dropping null ids')

    #drop duplicates
    socio_data = socio_data.sort_values(by=['unique_user_id_autofilled_do_not_edit', 'timestamp'])
    socio_data.drop_duplicates(subset=['unique_user_id_autofilled_do_not_edit'], keep='last', inplace=True)
    print(len(socio_data),'surveys', socio_data['unique_user_id_autofilled_do_not_edit'].nunique(), 'users after dropping duplicates')

    #prepare survey ids for merging
    socio_data['user_id_socio'] = socio_data['unique_user_id_autofilled_do_not_edit'].astype(str)
    socio_data['user_id_socio'] = socio_data['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
    socio_data['user_id_socio'] = socio_data['user_id_socio']
    socio_data = socio_data.drop(labels='unique_user_id_autofilled_do_not_edit', axis=1)
    
    
    #prepare trip ids for merging
    trips['user_id_socio'] = trips.perno.astype(str)
    trips['user_id_socio'] = trips['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
    trips.user_id_socio = [i.replace('-','') for i in trips.user_id_socio] # remove all dashes from strings
    
    #merge the data
    data = trips.merge(socio_data, on='user_id_socio')
    print(len(data), 'trips after merging')
    print(data.user_id_socio.nunique(), 'people after merging')
    
    data['program'] = program.split('_')[0]
    
    #add to list of datasets
    datasets.append(data)

In [ ]:
#merge them all together
full_data = pd.concat(datasets)
print(len(full_data), 'trips')
print(full_data.perno.nunique(), 'users')

In [ ]:
#filter out unlabeled trips -- accept partial labels -- needed for proper user count
labeled_data = full_data[full_data.data_user_input_mode_confirm.notna() | 
                         full_data.data_user_input_purpose_confirm.notna() |
                         full_data.data_user_input_replaced_mode.notna()]

print(len(labeled_data), 'labeled trips')
print(labeled_data.user_id_socio.nunique(), 'users who labeled')

In [ ]:
labeled_data.rename(columns={'user_id_socio':'user_id',
                          'please_identify_which_category_represents_your_total_household_':'HHINC',
                          'how_many_motor_vehicles_are_owned_leased_or_available_for_regul':'VEH',
                            ' how_many_motor_vehicles_are_owned_leased_or_available_for_regul':'VEH',
                             'how_many_motor_vehicles_are_owned_leased_or_available_for_regul ':'VEH',
                           'in_which_year_were_you_born?':'AGE',
                          'including_yourself_how_many_people_live_in_your_home?':'HHSIZE',
                          'how_many_children_under_age_18_live_in_your_home?':'CHILDREN',
                          'what_is_your_gender?':'GENDER',
                          'if_you_were_unable_to_use_your_household_vehicles_which_of_the_':'available_modes',
                          'are_you_a_student?':'STUDENT',
                         'data_duration':'duration', 
                         'data_distance':'distance'}, inplace=True)

In [ ]:
data = labeled_data.copy()
programs = ["4c", "cc", "fc", "pc", "sc", "vail"]
returned_dfs = separate_and_count_programs(data, programs)

so far so good, we're looking for at least 122 users and at least 61,496 trips after ALL cleaning

In [ ]:
data = labeled_data.copy()

#first, add the cleaned mode
data['Mode_confirm']= data['data_user_input_mode_confirm'].map(dic_re)

#second, add the cleaned replaced mode ASSUMES PROGRAM
data['Replaced_mode']= data['data_user_input_replaced_mode'].map(dic_re)

#third, add the cleaned purpose
data['Trip_purpose']= data['data_user_input_purpose_confirm'].map(dic_pur)

In [ ]:
#save the data for later
data.to_csv(to_data_folder + "/expanded_ct.csv")

In [ ]:
# Get timestamp from known year/month/day aggregated to days
data.rename(columns={'data_start_local_dt_year':'year','data_start_local_dt_month':'month','data_start_local_dt_day':'day'}, inplace=True)
data['date_time'] = pd.to_datetime(data[['year','month','day']])

# Fix age (birth year to age)
data['AGE'] = 2022 - data['AGE']

# Number of workers (size of HH - kids)
data['WORKERS'] = data['HHSIZE'] - data['CHILDREN']

# Duration in minutes (hours to minutes)
data['duration'] = data['duration'] / 60

# duration in miles (meters to miles)
data['distance_miles'] = data['distance'] * 0.0006213712

# E-bike/not E-Bike variable
data['is_ebike'] = "E-Bike Trips"
data.loc[data['Mode_confirm']!="E-bike", 'is_ebike'] = "Non E-Bike Trips"

In [ ]:
print(len(data), "trips")
print(data.user_id.nunique(), "users")

#loose some users that did not give this information (and their trips)
#records that had ’prefer not to say’ as a response for household income, household vehicles, and other available modes
data = data[~data['HHINC'].isin(['Prefer not to say'])]
data = data[~data['VEH'].isin(['Prefer not to say / Prefiero no decir.'])]
data = data[~data['available_modes'].isin(['None', 'Prefer not to say'])]

print(len(data), "trips after dropping non responses")
print(data.user_id.nunique(), "users after dropping non responses")

data['HHINC_NUM'] = data.HHINC.replace(['Less than $24,999',
                                       '$25,000-$49,999',
                                       '$50,000-$99,999',
                                       '$100,000 -$149,999',
                                       '$150,000-$199,999',
                                       '$200,000 or more'], [12500,37500,75000,125000,175000,250000])

# Calculate average income per adult in the household
data['PINC'] = data['HHINC_NUM'] / data['WORKERS']

# Combine variable categories
data = data.replace('Gas Car, drove alone', 'Car')
data = data.replace('Gas Car, with others', 'Shared Car')
data = data.replace('Bikeshare', 'Shared Micromobility')
data = data.replace('Scooter share', 'Shared Micromobility')
data = data.replace('Regular Bike', 'Personal Micromobility')
data = data.replace('Skate board', 'Personal Micromobility')
data = data.replace('Train', 'Transit')
data = data.replace('Free Shuttle', 'Transit')
data = data.replace('Bus', 'Transit')
data = data.replace('Walk', 'Walk')
data = data.replace('Taxi/Uber/Lyft', 'Ridehail')
data = data.replace('Pilot ebike', 'E-Bike')

# Categorical type will include all days/modes in groupby even if there is no data for a particular tabulation
data.user_id = pd.Categorical(data.user_id)
data.date_time = pd.Categorical(data.date_time)
data.mode_confirm = pd.Categorical(data.data_user_input_mode_confirm, ordered=True, categories=np.unique(list(dic_re.keys())))

# Add order to categorical variables
data.HHINC = pd.Categorical(data.HHINC, ordered=True)
data['Mode'] = pd.Categorical(data.Mode_confirm, ordered=True, categories=[
    'E-bike',
    'Car',
    'Shared Car',
    'Walk',
    'Transit',
    'Personal Micromobility',
    'Shared Micromobility',
    'Ridehail',
    'Other'])
data.VEH = data.VEH.astype(str)
data.VEH = pd.Categorical(data.VEH, ordered=True, categories=['0','1','2','3','4+'])
data['PINC_NUM'] = data['PINC']
data.PINC = pd.cut(data.PINC, bins=[0,10000,20000,30000,40000,50000,60000,70000,999999],
                  labels=["$0-9",
                         "$10-19",
                         "$20-29",
                         "$30-39",
                         "$40-49",
                         "$50-59",
                         "$60-69",
                         "$70+"])

In [ ]:
#filtered out ages that were greater than 100
data = data[data['AGE'] < 100]

#filter out durations longer than 8 hours
data = data[data['duration']<480]

#distances more than 50 miles 
data = data[data['distance_miles']<50]

#filter household sizes smaller than the number of kids
data = data[data['HHSIZE']>data['CHILDREN']]

#filter out households greater than 10
data = data[data['HHSIZE']<10]

In [ ]:
# Vehicles per driver
data = data[data['VEH'].notna()] #vails VEH nums were not strings?
data['VEH_num'] = data['VEH'].replace(['1','2','3','4+'],[1,2,3,4]).astype(int)
data['DRIVERS'] = data["including_yourself_how_many_people_have_a_driver's_license_in_y"]
data['DRIVERS_num'] = data['DRIVERS'].replace
data['veh_per_driver'] = (data['VEH_num'] / data['DRIVERS']).fillna(0)
data.loc[data['veh_per_driver']==np.inf, 'veh_per_driver'] = 0

#filter out 'not a trip' trips
data = data[~data['Mode_confirm'].isin(['Not a Trip'])]
data = data[~data['Replaced_mode'].isin(['Not a Trip'])]
data = data[~data['Trip_purpose'].isin(['not_a_trip'])]

print(len(data), 'trips after filtering') #around 63,000
print(data.user_id.nunique(), 'users after filtering') #132

# filtering out trips before first e-bike

In [ ]:
data.rename(columns = {'data_start_ts':'start_ts'}, inplace=True)

programs = ["4c", "cc", "fc", "pc", "sc", "vail"]
program_dfs = separate_and_count_programs(data, programs)

#filtering each of them
from datetime import datetime

after_ebike_dfs = []
for program_df in program_dfs:
    print("processing", program_df.program.unique())
    after_first_ebike_trips = filter_before_ebike(program_df)
    after_ebike_dfs.append(after_first_ebike_trips)
    
#combining the filtered datasets
filtered_merged = pd.concat(after_ebike_dfs, axis=0)

#check number of trips and users
separate_and_count_programs(filtered_merged, programs)

print(len(filtered_merged), "trips in combined df") #
print(filtered_merged['user_id'].nunique(), "users in combined df") #

In [ ]:
#Summary statistics table
stat_data = filtered_merged[['distance','duration']]
stat_data.describe()

In [ ]:
#save as a csv, to be used as input to analysis!
filtered_merged.to_csv(to_data_folder + "/tsdc_filtered_merged_trips.csv")